In [ ]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
wallet.import_key('test', '5JMXaLz5xnVvwrnvAGaZKQZFCDdeU6wjmuJY1rDnXiUZz7Gyi1o')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

In [ ]:
eosapi.get_public_key('5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

In [ ]:
eosapi.get_public_key('5JMXaLz5xnVvwrnvAGaZKQZFCDdeU6wjmuJY1rDnXiUZz7Gyi1o')

In [ ]:
eosapi.get_balance('helloworld11')

In [ ]:
key1 = 'EOS7ZC3YFTCpPknEniq5xiHxHBt2tSW2E5eMXEcxHDD54UBWEgN4G'
key2 = 'EOS7ZC3YFTCpPknEniq5xiHxHBt2tSW2E5eMXEcxHDD54UBWEgN4G'
eosapi.create_account('helloworld11', 'helloworld33', key1, key2, 64*1024, 1.0, 1.0)

In [23]:
code = open('/Users/newworld/dev/eos/build/contracts/test_vm/test_vm.wasm', 'rb').read()
abi = open('/Users/newworld/dev/eos/build/contracts/test_vm/test_vm.abi', 'r').read()
eosapi.set_contract('helloworld33', code, abi, 0)

{'transaction_id': 'c507e73112b71c163d2a8a28dfcee4a09ff31e1673d9c42d11301f2a36c8d402',
 'processed': {'id': 'c507e73112b71c163d2a8a28dfcee4a09ff31e1673d9c42d11301f2a36c8d402',
  'block_num': 4935,
  'block_time': '2019-06-21T02:39:28.500',
  'producer_block_id': None,
  'receipt': {'status': 'executed',
   'cpu_usage_us': 695,
   'net_usage_words': 338},
  'elapsed': 695,
  'net_usage': 2704,
  'scheduled': False,
  'action_traces': [{'receipt': {'receiver': 'eosio',
     'act_digest': '4512fad21aed9f58af75e35c716e8fcc18a6db3efb481de8bca37e4977c31688',
     'global_sequence': 5036,
     'recv_sequence': 4988,
     'auth_sequence': [['helloworld33', 5]],
     'code_sequence': 1,
     'abi_sequence': 1},
    'act': {'account': 'eosio',
     'name': 'setcode',
     'authorization': [{'actor': 'helloworld33', 'permission': 'active'}],
     'data': {'account': 'helloworld33',
      'vmtype': 0,
      'vmversion': 0,
      'code': '0061736d0100000001260660017e0060077e7e7e7e7e7f7f0060027f7f01

In [24]:
code = '''
def apply(receiver, code, action):
    ret = call_contract('helloworld33', 'calltest1', 0, 0, 0, int.to_bytes(1, 8, 'little'))
    print(ret)
    print(int.from_bytes(ret, 'little'))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

+++++++++++call: extra_args:1
b'\x02\x00\x00\x00\x00\x00\x00\x00'
2

